In [1]:
# 모듈 load
import pandas as pd
import numpy as np
import json

import torch
from tokenization_kobert import KoBertTokenizer
from transformers import AutoModelForSequenceClassification

In [2]:
import time
import re

In [3]:
from ast import literal_eval

In [4]:
# intent_list 생성(MRC 전용)
a = open("data/2차/intents_mrc용.txt", "r", encoding="utf-8")

intent_list = literal_eval(a.read())

del a

In [8]:
# dialogue 읽기

with open('data/2차/dialog_nodes.json', 'r',encoding="utf-8") as f2:
    json_data2 = json.load(f2)

In [9]:
# dialogue_intents에 해당하는 conditions이 존재하는 dialogue만 뽑아보자(처음 DIA() 에만 쓸거임)
json_data3 = []
for x in json_data2:
    try:
        if x["conditions"][0] == "#":
            json_data3.append(x)
    except:
        pass

In [12]:
# title 전체 목록 읽기
title_list = []
a = open("data/2차/title.txt","r",encoding="utf-8")
for x in a.readlines():
    title_list.append(x[:-1])

In [14]:
# gpu 메모리 청소
torch.cuda.empty_cache()

# device = gpu 설정
if torch.cuda.is_available():     
    device = torch.device("cuda")
    print('%d개 존재함' % torch.cuda.device_count())
    print('사용할 GPU : ', torch.cuda.get_device_name(0))

# If not...
else:
    print('GPU 없어서 CPU로 설정')
    device = torch.device("cpu")

1개 존재함
사용할 GPU :  NVIDIA GeForce RTX 3060


In [15]:
# 모델 load
model2 = AutoModelForSequenceClassification.from_pretrained("../model/kobert").cuda()

# 토크나이저 선언
tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert")

# load 파라미터(변경시 꼭 수정해야해!)
max_len = 152

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [16]:
# 모델 돌려돌려
def MRC(input_ids,attention_masks,start):
    model2.eval()

    predictions = []
    last_layer_attentions = []

    input_ids = input_ids.to(device)
    attention_masks = attention_masks.to(device)

    for i in range(len(input_ids)):
        ids = input_ids[i].unsqueeze(0)
        masks = attention_masks[i].unsqueeze(0)

        with torch.no_grad():
            outputs = model2(input_ids=ids,
                            attention_mask=masks)

        logits = outputs[0]
        logits = torch.softmax(logits,dim=1)
        last_layer_attention = outputs[1][-1]

        logits = logits.detach().cpu().numpy()
        last_layer_attention = last_layer_attention.detach().cpu().numpy()

        last_layer_attentions.append(last_layer_attention) 
        predictions.append(logits)

    probs = predictions[0][0]
    pred_idx = np.argmax(probs)

    return intent_list[pred_idx]

In [18]:
# DIA, DIA2 에 있는 if start2 == "처음으로" 이거는 나중에 intents리스트에 처음으로 라는 의도
# 가 존재하면 발동되는 구문임. 만약 그냥 처음으로 들으면 처음으로 돌리고 싶으면 챗봇 시작 셀을
# 수정하면 된다

In [19]:
# intents에 해당하는 다이얼로그 불러오고 상태 출력
# start2 = 배송진행상황조회

def DIA(start2):
    if start2 == "처음으로": # 처음이면 
        time.sleep(0.5)
        print("다시 돌아가겠습니다.")
        return None
        
    for x in json_data3: # intents = conditions 인 다이얼로그만 들어있는 리스트에서 찾기
        if x["conditions"] == "#"+start2: # 일단은 condition 기준으로 만들고자함
            #  만약 선택지면 next_step이 없음을 이용함
            try:
                if x["output"] == {}: # 연결은 되어있는데 output만 없는 경우 있음
                    ending_temp = ["-계속해서-","move",x["next_step"]["dialog_node"]]
                else:
                    ending_temp = [x["output"]["text"]["values"][0],"move",x["next_step"]["dialog_node"]]
            except:
                ending_temp = [x["output"]["text"]["values"][0],"select",x["title"]]
            return ending_temp

# 만약 아예 모르겠다 하면 ending_temp 는 None으로 리턴한다

In [28]:
# intents은 이미 파악했고 연결을 해줌
# 배송산간지역배송,ending
def DIA2(start2,ending_temp):
    if start2 == "처음으로":
        time.sleep(0.5)
        print("다시 돌아가겠습니다.")
        return None

    if ending_temp[1] == "YNO":
        if ending_temp[2][-2] == "-": # - 인경우는 뒤에 바로 붙기때문에 꼼수를 써야함
            ending_temp[2] = ending_temp[2] + "ex" # len을 2 늘린다!
        # start2에 따라 이동할 노드를 저장해야함
        # 그 다음 선택지도 선택지일수도 있고 아닐수도있음 -> 확인해서 [1]을 수정해놔야한다!
        if start2 == "네":
            for x in json_data2:
                # title이 없는 경우도 있어서 오류남(예시 - 회원가입)
                try:
                    if x["title"] is not None:
                        pass
                except:
                    continue

                # title 기준
                if x["title"] == ending_temp[2][:-2]+"-Y":
                    try:
                        ending_temp = [x["output"]["text"]["values"][0],"move",x["next_step"]["dialog_node"]]
                    except:
                        ending_temp = [x["output"]["text"]["values"][0],"select",x["title"]]
                    return ending_temp

        elif start2 == "아니요":
            for x in json_data2:
                # title 기준
                if x["title"] == ending_temp[2][:-2]+"-N":
                    try:
                        ending_temp = [x["output"]["text"]["values"][0],"move",x["next_step"]["dialog_node"]]
                    except:
                        ending_temp = [x["output"]["text"]["values"][0],"select",x["title"]]
                    return ending_temp
        else: # elif start2 == "몰라요": mrc 이용해서 잘못된 응답임을 확인하던가
            for x in json_data2:
                # title 기준
                if x["title"] == ending_temp[2][:-2]+"-O":
                    try:
                        ending_temp = [x["output"]["text"]["values"][0],"move",x["next_step"]["dialog_node"]]
                    except:
                        ending_temp = [x["output"]["text"]["values"][0],"select",x["title"]]
                    return ending_temp

    elif ending_temp[1] == "AB":
        if ending_temp[2][-2] == "-": # - 인경우는 뒤에 바로 붙기때문에 꼼수를 써야함
            ending_temp[2] = ending_temp[2] + "ex"
        pass # 나중에 구현


    elif ending_temp[1] == "move": # next_step이 있으므로 dialog_node와 title이 안맞는 경우 발생
        # ending_temp[2] == "node_5_1510637060394" 라면?
        if ending_temp[2] == "node_5_1510637060394":
            time.sleep(0.5)
            print("다시 돌아가겠습니다.")
            return None
        
        # 그래서 dialog_node로 비교해야함
        for x in json_data2:
            # dialog_node 기준
            if x["dialog_node"] == ending_temp[2]:
                try:
                    if x["output"] == {}:
                        ending_temp = ["-계속해서-","move",x["next_step"]["dialog_node"]]
                    else:
                        ending_temp = [x["output"]["text"]["values"][0],"move",x["next_step"]["dialog_node"]]
                except:
                    ending_temp = [x["output"]["text"]["values"][0],"select",x["title"]]
                return ending_temp

# 만약 아예 모르겠다 하면 ending_temp 는 None으로 리턴한다


In [29]:
# intents에 해당하는 슬롯 불러오기
# 미정

In [32]:
### 시작2 ###
okay = 0

text = "안녕하세요 동원 챗봇 푸디입니다. 언제든 말씀해 주세요."
while True:
    # 처음
    if okay == 0:
        start = input(text)

        # 종료시
        if start == "종료":
            break

        # 토큰화 하기
        input_ids = []
        attention_masks = []

        ## 읽고 토큰화하기
        encoded_dict = tokenizer.encode_plus(
                                start,            
                                add_special_tokens = True,
                                max_length = max_len,
                                pad_to_max_length = True, # padding =True 나 padding="longest" 같은걸로 대체하세요
                                return_attention_mask = True,
                                return_tensors = 'pt')
        
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

        input_ids = torch.cat(input_ids, dim=0)
        attention_masks = torch.cat(attention_masks, dim=0)

        # mrc 돌리기
        start2 = MRC(input_ids,attention_masks,start)
        okay = 1 # 2번쨰 전환
        ending = DIA(start2) # [문장,성격,title or dialog_node]

        if ending == None:
            time.sleep(0.5)
            print("이해하기 어려워요. 쉽게 얘기해주세요")
            okay = 0
            continue
        
    # 2번째부터
    if okay == 1:
        if start == "종료":
            break

        if ending == None:
            time.sleep(0.5)
            okay = 0
            continue

        start = input(ending[0])

        if start == "종료":
            break

        if ending == None:
            time.sleep(0.5)
            okay = 0
            continue

        if start == "처음으로":
            time.sleep(0.5)
            print("다시 돌아가겠습니다.")
            okay = 0
            continue

        # ending = [문장,성격,title or dialog_node]
        # 확인할때 유용한 print
        # print(ending)

        ## select 경우, (이미 답변을 준 상태 -> 답변에 해당하는 노드로 이동시켜줘야함)
        if ending[1] == "select":
            # title뒤에 어떤 글자가 들어가냐만을 파악
            for x in title_list:
                if ending[2][-2] == "-":
                    if x == ending[2]+"-Y":
                        ## Y, N ,O 라는 뜻
                        ending[1] = "YNO"
                        ending = DIA2(start,ending)
                        break
                elif ending[2][-2] == "_":
                    if x == ending[2][:-2]+"-Y":
                        ## Y, N ,O 라는 뜻
                        ending[1] = "YNO"
                        ending = DIA2(start,ending)
                        break
                # A,B는 아직 미구현(YNOA 인 경우도 있고 복잡함..)
                # elif x == ending[2][:-2]+"-A": # [:len(ending[2])]
                #     ## A,B 라는 뜻
                #     ending[1] = "AB"
                #     ending = DIA2(start,ending)
                #     break 
        ## move 경우,
        elif ending[1] == "move":
            ending = DIA2(start,ending)

다시 돌아가겠습니다.


'회원가입혜택안내'